# Local Oscillator Phase Noise

In [ ]:
import torchsig.transforms.functional as F

import numpy as np
import scipy as sp
%matplotlib inline
import matplotlib.pyplot as plt

Generates a tone-based test signal and applies the LO phase noise impairment.

In [ ]:
sample_rate = 10e6
rng = np.random.default_rng(42)

N = 8192

n = np.arange(0,N)
t = n/sample_rate
center_frequency = 0
#center_frequency = sample_rate/N
tone_bb_data = np.exp(2j*np.pi*center_frequency*t)

phase_noise_std = 100

data_out = F.local_oscillator_phase_noise(
    data = tone_bb_data,
    sample_rate = sample_rate,
    phase_noise_std = phase_noise_std,
    rng = rng
)


Time-domain plots show how the phase noise effects the complex sinusoid input.

In [ ]:

fig = plt.figure(figsize=(10,6))
fig.subplots_adjust(hspace=0.5)
ax = fig.add_subplot(2,1,1)
ax.plot(t,np.real(tone_bb_data),label='Real, Input Tone')
ax.plot(t,np.imag(tone_bb_data),label='Imag, Input Tone')
ylim = np.max(np.abs(tone_bb_data))*1.1
ax.set_ylim([-ylim,ylim])
ax.set_xlim([t[0],t[-1]])
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.grid()
ax.legend(loc='upper right')

ax = fig.add_subplot(2,1,2)
ax.plot(t,np.real(data_out),label='Real, Tone with Phase Noise')
ax.plot(t,np.imag(data_out),label='Imag, Tone with Phase Noise')
ax.set_ylim([-ylim,ylim])
ax.set_xlim([t[0],t[-1]])
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.grid()
ax.legend(loc='upper right')



Plots the frequency domain response, demonstrating the frequency drift effect.

In [ ]:
win = sp.signal.windows.blackmanharris(len(tone_bb_data))
fft_size = 2**20
f = np.linspace(-0.5,0.5-(1/fft_size),fft_size)*sample_rate
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1)
ax.plot(f,20*np.log10(np.abs(np.fft.fftshift(np.fft.fft(tone_bb_data*win,fft_size)))),label='Input Tone')
ax.plot(f,20*np.log10(np.abs(np.fft.fftshift(np.fft.fft(data_out*win,fft_size)))),'--',label='Output Tone with Phase Noise')
ax.legend(loc='upper right')
ax.set_xlim([-100000+center_frequency,100000+center_frequency])
ax.grid()
ax.set_ylabel('Magnitude (dB)')
ax.set_xlabel('Frequency (Hz)')


Plots the instantaneous phase of the output, showing the phase noise effect.

In [ ]:
frequency = np.unwrap(np.angle(data_out))*(sample_rate/(2*np.pi))
phase = np.diff(frequency)-center_frequency

fig = plt.figure(figsize=(10,6))
fig.subplots_adjust(hspace=0.5)
ax = fig.add_subplot(2,1,1)
ax.plot(t,frequency)
ax.set_title('Instantaneous Frequency of Signal with LO Phase Noise')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Frequency (Hz)')
ax.grid()

ax = fig.add_subplot(2,1,2)
ax.plot(t[0:1000],phase[0:1000])
ax.plot(t[0:1000],phase_noise_std*np.ones(1000),'r',label='Phase Noise Standard Deviation')
ax.plot(t[0:1000],-phase_noise_std*np.ones(1000),'r')
ax.set_title('Instantaneous Phase of Signal with LO Phase Noise')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Frequency (Hz)')
ax.grid()
ax.legend(loc='upper right')
